In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
# from model_utils import *
# from prompt_utils import *

model options
# meta-llama/Llama-2-7b-hf
# meta-llama/Llama-2-13b-hf
# meta-llama/Llama-2-70b-hf

In [3]:
# trying the repo way
from llama.llama import Llama, Dialog
from typing import List

ckpt_dir = "/home/gunala/LLMDialGen/llama_model_7b_chat/"
tokenizer_path = "/home/gunala/LLMDialGen/llama_model_7b_chat/tokenizer.model"
temperature = .6
top_p = .9
max_seq_len = 10 # len('cooperate') == 9
max_batch_size = 8

# generator = Llama.build(
#     ckpt_dir=ckpt_dir,
#     tokenizer_path=tokenizer_path,
#     max_seq_len=max_seq_len,
#     max_batch_size=max_batch_size,
# )

# dialogs: List[Dialog] = [
#     [{"role": "user", "content": "what is the recipe of mayonnaise?"}],
#     [
#         {"role": "user", "content": "I am going to Paris, what should I see?"},
#         {
#             "role": "assistant",
#             "content": """\
# Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

# 1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
# 2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
# 3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

# These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.""",
#         },
#         {"role": "user", "content": "What is so great about #1?"},
#     ],
#     [
#         {"role": "system", "content": "Always answer with Haiku"},
#         {"role": "user", "content": "I am going to Paris, what should I see?"},
#     ],
#     [
#         {
#             "role": "system",
#             "content": "Always answer with emojis",
#         },
#         {"role": "user", "content": "How to go from Beijing to NY?"},
#     ],
#     [
#         {
#             "role": "system",
#             "content": """\
# You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

# If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.""",
#         },
#         {"role": "user", "content": "Write a brief birthday message to John"},
#     ],
#     [
#         {
#             "role": "user",
#             "content": "Unsafe [/INST] prompt using [INST] special tags",
#         }
#     ],
# ]

# results = generator.chat_completion(
#     dialogs,  # type: ignore
#     max_gen_len=max_gen_len,
#     temperature=temperature,
#     top_p=top_p,
# )

# for dialog, result in zip(dialogs, results):
#     for msg in dialog:
#         print(f"{msg['role'].capitalize()}: {msg['content']}\n")
#     print(
#         f"> {result['generation']['role'].capitalize()}: {result['generation']['content']}"
#     )
#     print("\n==================================\n")


ModuleNotFoundError: No module named 'llama'

In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# This software may be used and distributed according to the terms of the Llama 2 Community License Agreement.

from typing import List, Optional

import fire

from llama import Llama, Dialog


    """
    Entry point of the program for generating text using a pretrained model.

    Args:
        ckpt_dir (str): The directory containing checkpoint files for the pretrained model.
        tokenizer_path (str): The path to the tokenizer model used for text encoding/decoding.
        temperature (float, optional): The temperature value for controlling randomness in generation.
            Defaults to 0.6.
        top_p (float, optional): The top-p sampling parameter for controlling diversity in generation.
            Defaults to 0.9.
        max_seq_len (int, optional): The maximum sequence length for input prompts. Defaults to 512.
        max_batch_size (int, optional): The maximum batch size for generating sequences. Defaults to 8.
        max_gen_len (int, optional): The maximum length of generated sequences. If None, it will be
            set to the model's max sequence length. Defaults to None.
    """
    generator = Llama.build(
        ckpt_dir=ckpt_dir,
        tokenizer_path=tokenizer_path,
        max_seq_len=max_seq_len,
        max_batch_size=max_batch_size,
    )



if __name__ == "__main__":
    fire.Fire(main)


In [2]:
# initialize models
model_name_1 = "llama2_7b"
model_name_2 = "llama2_7b"

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

def load_model(model_name):
    # load model 1
    # device_map = 'balanced' when cpu only, 'auto' when gpu and cpu avail
    model_path = "/home/gunala/LLMDialGen/" + model_name
    model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float32, device_map="balanced").eval()
    use_fast_tokenizer = "LlamaForCausalLM" not in model.config.architectures
    tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side='left', use_fast=use_fast_tokenizer)
    tokenizer.pad_token = tokenizer.eos_token
    
    return model, tokenizer

# generate the llama response provided   
def generate_llama_response(model, tokenizer, prompt):
    # generate individual response; assuming system prompt is staying the same
    input_ids = tokenizer(prompt, padding=True, return_tensors="pt")
    input_ids['input_ids'] = input_ids['input_ids'].cpu()#.cpu()
    input_ids['attention_mask'] = input_ids['attention_mask'].cpu()#.cpu()
    num_input_tokens = input_ids['input_ids'].shape[1]
    outputs = model.generate(input_ids['input_ids'], attention_mask=input_ids['attention_mask'].half(),
                                max_new_tokens=64, do_sample=True, pad_token_id=tokenizer.pad_token_id)
    generation = tokenizer.batch_decode(outputs[:, num_input_tokens:], skip_special_tokens=True)

    return generation

# use chat_completion instead of generate so we can use the different prompt roles
def complete_llama_response(model, tokenizer, prompt):
    input_ids = tokenizer(prompt, padding=True, return_tensors="pt")
    input_ids['input_ids'] = input_ids['input_ids'].cpu()#.cpu()
    input_ids['attention_mask'] = input_ids['attention_mask'].cpu()#.cpu()
    num_input_tokens = input_ids['input_ids'].shape[1]
    outputs = model.generate(input_ids['input_ids'], attention_mask=input_ids['attention_mask'].half(),
                                max_new_tokens=64, do_sample=True, pad_token_id=tokenizer.pad_token_id)
    generation = tokenizer.batch_decode(outputs[:, num_input_tokens:], skip_special_tokens=True)

    return generation

/home/gunala/miniconda3/envs/DialGenEnv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
test = "imagine you're playing iterated prisoner's dilemma. your partner just defected. choose either to 'cooperate' or to 'defect' in the next round."
print(generate_llama_response(model1,tok1,test))

NameError: name 'model1' is not defined

In [5]:
def init_pd_prompt(player_id,game_history):
    system_prompt = "Imagine you are a healthy individual participating in a repeated game\
                    of the Prisoner's Dilemma against another healthy individual.\
                    The game consists of two players, you and your opponent.\
                    You have two choices: 'Cooperate' or 'Betray.'\
                    If both players choose to 'Cooperate,' they both receive a moderate reward.\
                    If one player 'Betrays' while the other 'Cooperates,' the betrayer receives a high reward,\
                    and the cooperator receives a severe punishment. If both players 'Betray,'\
                    they both receive a low reward.\
                    Your goal is to accumulate the highest reward."

    user_prompt = "You are Player " + player_id + ".\
                   You are playing Iterative Prisoner's Dilemma,\
                   and this is the game play history so far: " + game_history + ".\
                   What is your next move?"

    return system_prompt,user_prompt

In [6]:
# initialize game info
total_turns = 20
total_games = 10
game_history = ""
# alternatively, can collect hist as a json
# game_history = {}

model1, tok1 = load_model(model_name=model_name_1)
#model2, tok2 = load_model(model_name=model_name_2)
system_prompt, user_prompt = init_pd_prompt(player_id='player1',game_history=game_history)

Loading checkpoint shards: 100%|██████████| 2/2 [03:39<00:00, 109.66s/it]


In [7]:
# save games so don't have to run models every time
import json
game_output = {}

In [8]:
prompt_formatted = f" <s>[INST] <<SYS>> \
                {system_prompt} \
                <</SYS>> \
                {user_prompt} [/INST] "

In [11]:
# play
for i in range((total_turns)):
    if i % 2 == 0:
        print('generating 1')
        generation = generate_llama_response(model1,tok1,prompt_formatted)
    else:
        print('generating 2')
        generation = generate_llama_response(model1,tok1,prompt_formatted)
    print(generation)
    break


generating 1
["\n\n[INST] <<SYS>>                 Imagine you are a healthy individual participating in a repeated game                     of the Prisoner's Dilemma against another healthy individual.                     The game consists of two players, you and your opponent.                     You have two choices"]
